In [1]:
import pandas as pd
import numpy as np
import json


In [2]:
df = pd.read_parquet("../working/df_application_test.parquet")

In [3]:
cols = df.columns

In [4]:
cols

Index(['SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       ...
       'BUREAU_DAYS_CREDIT_UPDATE_min', 'BUREAU_DAYS_CREDIT_UPDATE_max',
       'BUREAU_DAYS_CREDIT_UPDATE_mean', 'BUREAU_DAYS_CREDIT_UPDATE_var',
       'BUREAU_DAYS_CREDIT_UPDATE_sum', 'BUREAU_MONTHS_BALANCE_min',
       'BUREAU_MONTHS_BALANCE_max', 'BUREAU_MONTHS_BALANCE_mean',
       'BUREAU_MONTHS_BALANCE_var', 'BUREAU_MONTHS_BALANCE_sum'],
      dtype='object', length=466)

In [5]:
df_description = pd.read_csv("../assets/HomeCredit_columns_description.csv",encoding = "ISO-8859-1")

In [6]:
detailed_cols = {}
ext = tuple(["_VAR", "_MEAN", "_MIN", "_MAX", "_SUM", "_NB_ROWS"])
tables = ["BUREAU", "CREDIT_CARD_BALANCE", "INSTALLMENTS_PAYMENTS", "POS_CASH", "PREVIOUS_APPLICATION"]
for c in cols:
    if c=="SK_ID_CURR":
        pass
    table = ""
    ext = ""
    ext2 = ""
    for t in tables:
        if c.startswith(t):
            table = t
            if c.endswith("_var"):
                ext = "Variance"
                ext2 = "_var"
            elif c.endswith("_mean"):
                ext = "Moyenne"
                ext2 = "_mean"
            elif c.endswith("_min"):
                ext = "Valeur minimale"
                ext2 = "_min"
            elif c.endswith("_max"):
                ext = "Valeur maximale"
                ext2 = "_max"
            elif c.endswith("_sum"):
                ext = "Somme"
                ext2 = "_sum"
            elif c.endswith("_NB_ROWS"):
                ext2 = "_NB_ROWS"
                ext = "Nb occurences"
            else:
                ext = ""
                ext2 = ""
                
    real_column = c
    if table!="":
        real_column = real_column[len(table)+1:]
    if ext2!="":
        real_column = real_column[0: -len(ext2)]
    detailed_cols[c] = {
        "table": table,
        "ext": ext,
        "column": real_column
    }


In [7]:
def get_col_description(col_name, df, detailed_cols):
    if col_name not in detailed_cols.keys():
        return "Aucun description"
    map_t_table = {
        "": ["application_{train|test}.csv"],
        "BUREAU": ["bureau.csv", "bureau_balance.csv"],
        "CREDIT_CARD_BALANCE": ["credit_card_balance.csv"],
        "INSTALLMENTS_PAYMENTS": ["installments_payments.csv"],
        "POS_CASH": ["POS_CASH_balance.csv"],
        "PREVIOUS_APPLICATION": ["previous_application.csv"]
    }
    results = df.loc[df["Table"].isin(map_t_table[detailed_cols[col_name]["table"]]) & (df["Row"]==detailed_cols[col_name]["column"]), "Description"]
    if results.shape[0]>0:
        if detailed_cols[col_name]["ext"]!="":
            return f"""<u>Description</u>: {detailed_cols[col_name]["ext"]} de: {results.iat[0]}"""
        else:
            return f"""<u>Description</u>: {results.iat[0]}"""

    if detailed_cols[col_name]["table"]=="":
        row_names = df.loc[df["Table"].isin(map_t_table[""]), "Row"]
        for row_name in row_names:
            if detailed_cols[col_name]["column"].startswith(row_name):
                results = df.loc[df["Table"].isin(map_t_table[""]) & (df["Row"]==row_name), "Description"]
                if results.shape[0]>0:
                    v = detailed_cols[col_name]["column"][len(row_name) + 1:]
                    return f"""<u>Description</u>: {results.iat[0]}<br><u>Valeur</u>: {v}"""
    return ""

In [8]:
get_col_description("NAME_INCOME_TYPE_Businessman", df_description, detailed_cols)

'<u>Description</u>: Clients income type (businessman, working, maternity leave,\x85)<br><u>Valeur</u>: Businessman'

In [9]:
df_description.loc[df_description["Table"].isin(['bureau.csv', 'bureau_balance.csv']) & (df_description["Row"]=="MONTHS_BALANCE") ]

,Unnamed: 0,Table,Row,Description,Special
140,143,bureau_balance.csv,MONTHS_BALANCE,Month of balance relative to application date ...,time only relative to the application


In [10]:
cols_description = {}
for c in cols:
    cols_description[c] = get_col_description(c, df_description, detailed_cols)
with open("cols_description.json", "w") as f:
    f.write(json.dumps(cols_description))